In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
file_path = r"C:\Users\7J6594897\Desktop\Github Projects\Clustering-CityofAustin311Calls\Data\Austin_311_Public_Data_20231107.csv"
data = pd.read_csv(file_path).sample(100)
data.head()
data

C:\Users\7J6594897\AppData\Local\Temp\ipykernel_34948\1399576220.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path).sample(100)


,Service Request (SR) Number,SR Description,Method Received,SR Status,Status Change Date,Created Date,Last Update Date,Close Date,SR Location,Street Number,...,Zip Code,County,State Plane X Coordinate,State Plane Y Coordinate,Latitude Coordinate,Longitude Coordinate,(Latitude.Longitude),Council District,Map Page,Map Tile
454264,21-00515933,Debris in Street,Phone,Resolved,11/19/2021 11:27:23 AM,11/19/2021 07:47:19 AM,11/19/2021 11:27:23 AM,11/19/2021 11:27:23 AM,"HOLLY SPRINGS DR & HOLLY SPRINGS CT, AUSTIN, TX",NaN,...,78748.0,TRAVIS,3.086117e+06,1.003786e+07,30.179423,-97.834441,"(30.17942296, -97.83444113)",5.0,672M,MD14
1272475,19-00254132,ARR Missed Yard Trimmings/Compost,Phone,Closed,07/08/2019 05:59:45 AM,07/05/2019 05:20:12 PM,07/08/2019 05:59:45 AM,07/08/2019 05:59:45 AM,"2604 MORENO ST, AUSTIN, TX 78723",2604,...,78723.0,TRAVIS,3.128469e+06,1.008048e+07,30.293876,-97.697249,"(30.293876, -97.69724896)",9.0,586B,ML24
499851,22-00281494,Wildlife Exposure,Phone,Closed,08/04/2022 05:19:08 PM,08/04/2022 03:26:44 PM,08/04/2022 05:19:08 PM,08/04/2022 05:19:08 PM,"11201 TRACTON LN, AUSTIN, TX 78739",11201.0,...,78739.0,TRAVIS,3.066063e+06,1.004047e+07,30.187823,-97.897709,"(30.18782263, -97.89770904)",8.0,671B,MA15
1476180,23-00080690,Tree Issue ROW,Phone,Resolved,02/10/2023 12:03:40 PM,02/10/2023 11:51:06 AM,02/10/2023 12:03:40 PM,02/10/2023 12:03:40 PM,"1914 TILLOTSON AVE, AUSTIN, TX 78702",1914.0,...,78702.0,TRAVIS,3.121794e+06,1.007042e+07,30.266668,-97.719140,"(30.26666812, -97.71913999)",1.0,585Y,MK22
1196736,19-00019349,Found Animal Report - Keep,Spot311 Interface,Closed,01/17/2019 06:09:42 PM,01/17/2019 03:51:13 PM,01/17/2019 06:09:42 PM,01/17/2019 06:09:42 PM,"12019 GATLING GUN LN, AUSTIN, TX 78748",12019,...,78748.0,TRAVIS,3.080121e+06,1.002832e+07,30.153573,-97.854083,"(30.15357338, -97.85408308)",8.0,702C,MD12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867679,18-00107651,Loose Dog,Phone,Closed,04/15/2018 03:13:52 PM,04/14/2018 02:07:59 PM,04/15/2018 02:13:54 PM,04/15/2018 03:13:52 PM,"8001 S IH 35 SVRD NB, AUSTIN, TX 78744",8001.0,...,78744.0,TRAVIS,3.103707e+06,1.003746e+07,30.177227,-97.778815,"(30.17722709, -97.77881467)",2.0,674K,MG14
1579221,23-00087466,ARR - Storm Debris Collection,Spot311 Interface,Duplicate (closed),05/19/2023 11:06:14 PM,02/13/2023 07:50:53 AM,05/19/2023 11:06:14 PM,05/19/2023 11:06:14 PM,"7703 SHADYROCK DR, AUSTIN, TX 78731",7703.0,...,78731.0,TRAVIS,3.108096e+06,1.010616e+07,30.365804,-97.759899,"(30.36580391, -97.75989919)",10.0,524D,MH30
1153367,18-00210394,ARR Missed Recycling,Phone,Closed,10/24/2018 01:37:49 PM,07/03/2018 04:05:28 PM,10/24/2018 01:37:43 PM,10/24/2018 01:37:49 PM,"224 LESSIN LN, AUSTIN, TX 78704",224,...,78704.0,TRAVIS,3.107979e+06,1.005565e+07,30.226948,-97.763970,"(30.22694791, -97.7639705)",3.0,614Y,MH19
331799,21-00309758,ARR Brush and Bulk,Phone,Closed,07/05/2021 06:13:15 AM,07/01/2021 01:24:28 PM,07/05/2021 06:13:15 AM,07/05/2021 06:13:15 AM,"3912 DRY CREEK DR, AUSTIN, TX 78731",3912.0,...,78731.0,TRAVIS,3.106131e+06,1.009862e+07,30.345202,-97.766681,"(30.34520226, -97.76668069)",10.0,524Q,MH29


In [9]:
# Selecting only the specified columns
selected_columns = [
    "Service Request (SR) Number", "SR Description"
    ]
data = data[selected_columns]

data.to_csv(r"C:\Users\7J6594897\Desktop\100samples.csv")
#Procceded to make modifications in Excel 

In [18]:
label_file_path = r"C:\Users\7J6594897\Desktop\100samples_labeled.csv"
labeled_data = pd.read_csv(label_file_path)
labeled_data.head()

,SR Description,Category
0,Debris in Street,Waste Collection and Recycling
1,ARR Missed Yard Trimmings/Compost,Tree Issues ans trimming
2,Wildlife Exposure,Animal Services
3,Tree Issue ROW,Tree Issues ans trimming
4,Found Animal Report - Keep,Animal Services


In [19]:
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.base import TransformerMixin

# Basic Text Preprocessor
class BasicTextPreprocessor(TransformerMixin):
    def transform(self, X, **transform_params):
        return [self.basic_preprocess(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def basic_preprocess(self, text):
        # Convert to lowercase
        text = text.lower()
        # Remove special characters
        text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
        # Simple tokenization (splitting text into words)
        tokens = text.split()
        return ' '.join(tokens)

model_data = labeled_data

# Split the data into training and testing sets
train_data, test_data = train_test_split(model_data, test_size=0.2, random_state=42)

# Create a pipeline with text preprocessing, TF-IDF vectorization, and SVM classifier
pipeline = make_pipeline(
    BasicTextPreprocessor(),
    TfidfVectorizer(),
    SVC(kernel='linear')  # Using a linear kernel for the SVM
)

# Train the model
pipeline.fit(train_data['SR Description'], train_data['Category'])

# Evaluate the model
predicted_categories = pipeline.predict(test_data['SR Description'])
report = classification_report(test_data['Category'], predicted_categories)

print(report)

                                        precision    recall  f1-score   support

                       Animal Services       0.86      1.00      0.92         6
                       Code Compliance       0.33      1.00      0.50         1
                         Public Safety       1.00      0.33      0.50         3
                  Shared Micromobility       1.00      1.00      1.00         1
                    Telecommunications       0.00      0.00      0.00         1
       Traffic and Road Infrastructure       1.00      1.00      1.00         1
Traffic and Road Infrastructure issues       0.00      0.00      0.00         0
        Waste Collection and Recycling       1.00      0.86      0.92         7

                              accuracy                           0.80        20
                             macro avg       0.65      0.65      0.61        20
                          weighted avg       0.87      0.80      0.80        20



c:\Users\7J6594897\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\7J6594897\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\7J6594897\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver

In [23]:
testing_data = pd.read_csv(file_path).sample(1000)
# Check if the column name for SR Descriptions in the new dataset is the same or adjust accordingly
# Predict the categories for the new dataset
new_data_predictions = pipeline.predict(testing_data['SR Description'])  # Replace 'SR Description' if column name is different

# Add the predictions as a new column to the new dataset
testing_data['Predicted Category'] = new_data_predictions

# Save the new dataset with predictions to a new file
testing_data.to_csv(r'C:\Users\7J6594897\Desktop\test100samples_labeled.csv', index=False)

C:\Users\7J6594897\AppData\Local\Temp\ipykernel_34948\1491904776.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  testing_data = pd.read_csv(file_path).sample(1000)
